In [1]:
import numpy as np
from pickle import load, dump
from pathlib import Path
from sklearn.cluster import KMeans
from tqdm import tqdm
from scipy.cluster.vq import vq
import torch

from kmeans import kmeans

NUM_CLUSTERS = 32 * 56

encoded_paths = list(Path("../../encoded/").glob("*.pk"))

data = None

for encode_path in tqdm(encoded_paths):
    with open(encode_path, "rb") as f:
        batch = load(f)
        batch["indxs"] = batch["indxs"].numpy()
    if data is None:
        data = batch
    else:
        data["x"] = np.concatenate([data["x"], batch["x"]])
        data["indxs"] = np.concatenate([data["indxs"], batch["indxs"]])
        data["image_paths"] += batch["image_paths"]

print(data["x"].shape)


100%|█████████████████████████████| 400/400 [00:02<00:00, 137.55it/s]

(48776, 512)


In [2]:
data["x"] = data["x"].astype(np.float32)
x = torch.from_numpy(data["x"])
# # scale
# m = x.mean(0, keepdim=True)
# s = x.std(0, unbiased=False, keepdim=True)
# x -= m
# x /= s
# # decomposition
# U, S, V = torch.linalg.svd(x)
# reduced = torch.mm(x, V[:, :32])
means, bins = kmeans(data["x"], num_clusters=NUM_CLUSTERS, num_iters=25)

# # kmeans = KMeans(NUM_CLUSTERS, verbose=1, max_iter=50)
# # kmeans.fit(data['x'])

# # with open(f"./kmeans_num-clusters={NUM_CLUSTERS}.pk", 'wb') as f:
# #     dump(kmeans, f)

with open(f"./torch-kmeans_num-clusters={NUM_CLUSTERS}-no-reduced.pk", "wb") as f:
    dump(means, f)




  0%|                                                                   | 0/25 [00:00<?, ?it/s]


RuntimeError: [enforce fail at alloc_cpu.cpp:66] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 179008700416 bytes. Error code 12 (Cannot allocate memory)

In [6]:

with open(f"../../torch-kmeans_num-clusters={NUM_CLUSTERS}-no-reduced.pk", "rb") as f:
    means = load(f)
    means = means.numpy()

# with open(f"./kmeans_num-clusters={NUM_CLUSTERS}.pk", 'rb') as f:
#     kmeans = load(f)
#     means = kmeans.cluster_centers_

print("Loaded means")

means_sorted = np.sort(means, axis=0)

closest, distances = vq(means_sorted, x)
print(closest)
print(closest.shape)
print("Computed distances")

Loaded means


ValueError: obs and code should have same number of features (columns)

In [2]:
from torchvision.utils import make_grid
import torchvision.transforms as T
from torchvision.transforms.functional import to_pil_image

from PIL import Image
from multiprocessing import Pool

tr = T.Compose([T.Resize((128, 128)), T.ToTensor()])


def read_and_transform(image_path):
    img = Image.open(image_path).convert("RGB")
    img = tr(img)
    return img


# with Pool() as pool:
x = list(
    tqdm(
        map(read_and_transform, [data["image_paths"][i] for i in closest]),
        total=len(closest),
    )
)
grid = make_grid(x, nrow=56)

to_pil_image(grid).save("./rf100-grid=32x56.png")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1792/1792 [00:06<00:00, 282.93it/s]
